# Data Exploration

**Author:** Nino Gagnidze
**Purpose:** Initial exploration and quality assessment of the Mall Customers dataset

## Objectives
- Load and inspect the raw dataset
- Understand data structure and types
- Identify data quality issues (missing values, duplicates, outliers)
- Generate initial statistical summaries

## 1. Setup and Data Loading

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

In [2]:
# Load the dataset
data_path = '../data/raw/mall_customers.csv'
df = pd.read_csv(data_path)

print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")

Dataset loaded successfully!
Dataset shape: (200, 5)


## 2. Initial Data Inspection

In [3]:
# Display first few rows
print("First 10 rows of the dataset:")
df.head(10)

First 10 rows of the dataset:


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
5,6,Female,22,17,76
6,7,Female,35,18,6
7,8,Female,23,18,94
8,9,Male,64,19,3
9,10,Female,30,19,72


In [4]:
# Display last few rows
print("Last 5 rows of the dataset:")
df.tail()

Last 5 rows of the dataset:


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18
199,200,Male,30,137,83


In [5]:
# Get dataset information
print("Dataset Information:")
df.info()

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [6]:
# Get column names and data types
print("Column Names and Data Types:")
print(df.dtypes)

Column Names and Data Types:
CustomerID                 int64
Gender                    object
Age                        int64
Annual Income (k$)         int64
Spending Score (1-100)     int64
dtype: object


## 3. Data Quality Assessment

In [7]:
# Check for missing values
print("Missing Values Count:")
print(df.isnull().sum())
print("\nMissing Values Percentage:")
print((df.isnull().sum() / len(df) * 100).round(2))

Missing Values Count:
CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

Missing Values Percentage:
CustomerID                0.0
Gender                    0.0
Age                       0.0
Annual Income (k$)        0.0
Spending Score (1-100)    0.0
dtype: float64


In [8]:
# Check for duplicate records
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    print("\nDuplicate rows:")
    print(df[df.duplicated(keep=False)])

Number of duplicate rows: 0


In [9]:
# Check for duplicate CustomerIDs
duplicate_ids = df['CustomerID'].duplicated().sum()
print(f"Number of duplicate CustomerIDs: {duplicate_ids}")

if duplicate_ids > 0:
    print("\nDuplicate CustomerIDs:")
    print(df[df['CustomerID'].duplicated(keep=False)].sort_values('CustomerID'))

Number of duplicate CustomerIDs: 0


## 4. Statistical Summary

In [10]:
# Generate descriptive statistics for numerical features
print("Descriptive Statistics:")
df.describe()

Descriptive Statistics:


,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
count,200.00,200.00,200.00,200.00
mean,100.50,38.85,60.56,50.20
std,57.88,13.97,26.26,25.82
min,1.00,18.00,15.00,1.00
25%,50.75,28.75,41.50,34.75
50%,100.50,36.00,61.50,50.00
75%,150.25,49.00,78.00,73.00
max,200.00,70.00,137.00,99.00


In [11]:
# Additional statistics
print("Additional Statistics:")
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

stats_df = pd.DataFrame({
    'Mean': df[numerical_cols].mean(),
    'Median': df[numerical_cols].median(),
    'Mode': df[numerical_cols].mode().iloc[0],
    'Std': df[numerical_cols].std(),
    'Variance': df[numerical_cols].var(),
    'Range': df[numerical_cols].max() - df[numerical_cols].min(),
    'IQR': df[numerical_cols].quantile(0.75) - df[numerical_cols].quantile(0.25)
})

stats_df

Additional Statistics:


,Mean,Median,Mode,Std,Variance,Range,IQR
CustomerID,100.50,100.5,1.0,57.88,3350.00,199,99.50
Age,38.85,36.0,32.0,13.97,195.13,52,20.25
Annual Income (k$),60.56,61.5,54.0,26.26,689.84,122,36.50
Spending Score (1-100),50.20,50.0,42.0,25.82,666.85,98,38.25


## 5. Categorical Features Analysis

In [12]:
# Analyze Gender distribution
print("Gender Distribution:")
print(df['Gender'].value_counts())
print("\nGender Percentage:")
print(df['Gender'].value_counts(normalize=True) * 100)

Gender Distribution:
Gender
Female    112
Male       88
Name: count, dtype: int64

Gender Percentage:
Gender
Female    56.0
Male      44.0
Name: proportion, dtype: float64


In [13]:
# Check for unique values in categorical columns
print("Unique values in Gender column:")
print(df['Gender'].unique())

Unique values in Gender column:
['Male' 'Female']


## 6. Outlier Detection (Initial Assessment)

In [14]:
# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column):
    """
    Detect outliers using the Interquartile Range (IQR) method.
    
    Parameters:
    -----------
    data : pd.DataFrame
        Input dataframe
    column : str
        Column name to check for outliers
    
    Returns:
    --------
    dict
        Dictionary with outlier information
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return {
        'column': column,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'outlier_count': len(outliers),
        'outlier_percentage': (len(outliers) / len(data)) * 100
    }

In [15]:
# Detect outliers for numerical columns
numerical_features = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']

print("Outlier Detection Summary (IQR Method):")
print("=" * 80)

for col in numerical_features:
    outlier_info = detect_outliers_iqr(df, col)
    print(f"\n{col}:")
    print(f"  Q1: {outlier_info['Q1']:.2f}")
    print(f"  Q3: {outlier_info['Q3']:.2f}")
    print(f"  IQR: {outlier_info['IQR']:.2f}")
    print(f"  Lower Bound: {outlier_info['lower_bound']:.2f}")
    print(f"  Upper Bound: {outlier_info['upper_bound']:.2f}")
    print(f"  Outlier Count: {outlier_info['outlier_count']}")
    print(f"  Outlier Percentage: {outlier_info['outlier_percentage']:.2f}%")

Outlier Detection Summary (IQR Method):

Age:
  Q1: 28.75
  Q3: 49.00
  IQR: 20.25
  Lower Bound: -1.62
  Upper Bound: 79.38
  Outlier Count: 0
  Outlier Percentage: 0.00%

Annual Income (k$):
  Q1: 41.50
  Q3: 78.00
  IQR: 36.50
  Lower Bound: -13.25
  Upper Bound: 132.75
  Outlier Count: 2
  Outlier Percentage: 1.00%

Spending Score (1-100):
  Q1: 34.75
  Q3: 73.00
  IQR: 38.25
  Lower Bound: -22.62
  Upper Bound: 130.38
  Outlier Count: 0
  Outlier Percentage: 0.00%


## 7. Data Range Validation

In [16]:
# Check if values are within expected ranges
print("Data Range Validation:")
print("=" * 80)

# Age validation (should be positive)
invalid_age = df[df['Age'] <= 0]
print(f"\nInvalid Age values (<=0): {len(invalid_age)}")

# Income validation (should be positive)
invalid_income = df[df['Annual Income (k$)'] <= 0]
print(f"Invalid Annual Income values (<=0): {len(invalid_income)}")

# Spending Score validation (should be between 1-100)
invalid_spending = df[(df['Spending Score (1-100)'] < 1) | (df['Spending Score (1-100)'] > 100)]
print(f"Invalid Spending Score values (not in 1-100 range): {len(invalid_spending)}")

# Gender validation
valid_genders = ['Male', 'Female']
invalid_gender = df[~df['Gender'].isin(valid_genders)]
print(f"Invalid Gender values: {len(invalid_gender)}")

Data Range Validation:

Invalid Age values (<=0): 0
Invalid Annual Income values (<=0): 0
Invalid Spending Score values (not in 1-100 range): 0
Invalid Gender values: 0


## 8. Initial Insights Summary

### Data Quality Status:
- **Missing Values:** 0 (No missing values found in any column)
- **Duplicate Records:** 0 (No duplicate rows identified)
- **Duplicate CustomerIDs:** 0 (All customer IDs are unique)
- **Data Types:** All correctly assigned (4 integer columns, 1 categorical column)

### Key Observations:
- **Total Records:** 200 customers
- **Total Features:** 5 (CustomerID, Gender, Age, Annual Income, Spending Score)
- **Gender Distribution:** 
  - Female: 112 customers (56%)
  - Male: 88 customers (44%)
- **Age Range:** 18 to 70 years (Mean: 38.85, Median: 36.0)
- **Annual Income Range:** $15k to $137k (Mean: $60.56k, Median: $61.5k)
- **Spending Score Range:** 1 to 99 (Mean: 50.2, Median: 50.0)

### Statistical Insights:
- Age has slight positive skewness (younger customers more common)
- Income distribution is relatively balanced around the median
- Spending scores are uniformly distributed across the range
- All data values fall within valid/expected ranges

### Data Quality Issues Identified:
1. **Outliers in Annual Income:** 2 customers (1%) with very high income (>$132.75k)
   - Decision: KEEP - These represent valid high-income customer segments
2. **No other data quality issues:** Data is clean and ready for analysis

### Next Steps:
1. No missing value imputation needed
2. No duplicate removal needed
3. Keep outliers as they represent valid customer segments
4. Proceed to feature engineering:
   - Create age groups (Young, Adult, Middle-Aged, Senior)
   - Create income categories (Low, Medium, High, Very High)
   - Create spending categories (Low, Medium, High)
   - Encode categorical variables for machine learning

## 9. Save Exploration Report

In [17]:
# Create a data quality report
quality_report = {
    'Total Records': len(df),
    'Total Features': len(df.columns),
    'Missing Values': df.isnull().sum().sum(),
    'Duplicate Rows': df.duplicated().sum(),
    'Duplicate CustomerIDs': df['CustomerID'].duplicated().sum(),
    'Gender Distribution': df['Gender'].value_counts().to_dict(),
    'Age Range': f"{df['Age'].min()} - {df['Age'].max()}",
    'Income Range': f"{df['Annual Income (k$)'].min()} - {df['Annual Income (k$)'].max()}",
    'Spending Score Range': f"{df['Spending Score (1-100)'].min()} - {df['Spending Score (1-100)'].max()}"
}

print("Data Quality Report:")
for key, value in quality_report.items():
    print(f"{key}: {value}")

Data Quality Report:
Total Records: 200
Total Features: 5
Missing Values: 0
Duplicate Rows: 0
Duplicate CustomerIDs: 0
Gender Distribution: {'Female': 112, 'Male': 88}
Age Range: 18 - 70
Income Range: 15 - 137
Spending Score Range: 1 - 99


In [18]:
# Save basic statistics to file for reference
stats_output_path = '../reports/results/data_exploration_stats.txt'

with open(stats_output_path, 'w') as f:
    f.write("DATA EXPLORATION SUMMARY\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Dataset Shape: {df.shape}\n")
    f.write(f"Total Records: {len(df)}\n")
    f.write(f"Total Features: {len(df.columns)}\n\n")
    f.write("Columns: " + ", ".join(df.columns.tolist()) + "\n\n")
    f.write("Data Types:\n")
    f.write(df.dtypes.to_string() + "\n\n")
    f.write("Missing Values:\n")
    f.write(df.isnull().sum().to_string() + "\n\n")
    f.write("Descriptive Statistics:\n")
    f.write(df.describe().to_string() + "\n")

print(f"Exploration summary saved to {stats_output_path}")

Exploration summary saved to ../reports/results/data_exploration_stats.txt
